In [1]:
import pandas as pd 
import os
from pulp import *

In [2]:
os.chdir("C:/Users/zuhre.b/OneDrive - Procter and Gamble/Desktop/IE313_Assignment1")

In [3]:
single70 = pd.read_excel("ProductInfo.xlsx",sheet_name='Single70')
double70 = pd.read_excel("ProductInfo.xlsx",sheet_name='Double70')
double80 = pd.read_excel("ProductInfo.xlsx",sheet_name='Double80')
total_const = pd.read_excel("ProductInfo.xlsx",sheet_name='Total Constraints')

In [4]:
#Max Overtime/RegularTime
max_overtime_const = 0.25

#Overtime cost increase
overtime_penalty = 1.5

In [5]:
#Initial Lists
months = list(single70['Months'])
products= list(["Single70","Double70","Double80"])

***Parameters***

In [6]:
#Product based parameters

prod_param = {}

for prod in products:
    if prod == "Single70":
        prod_param[prod] = {"StandartTime" : 1.65, "InitialInventory" : 6500}
    if prod == "Double70":
        prod_param[prod] = {"StandartTime" : 2.15, "InitialInventory" : 12000}
    if prod == "Double80":
        prod_param[prod] = {"StandartTime" : 2.68, "InitialInventory" : 3000}

In [7]:
#Time based parameters

CostRegularMan=list(total_const["Cost of Regular Man-Hour"])
AvailableManHours=list(total_const["Available Man-Hours"])
InventoryCarryRate=list(total_const["Inventory Carry Rate"])

month_param = {}

for i in range(len(months)):
    month_param[months[i]] = {"CostRegularMan" : CostRegularMan[i],"AvailableManHours" : AvailableManHours[i],"InventoryCarryRate" : InventoryCarryRate[i]}

In [8]:
#Time and Product based parameters

unitcosts70=list(single70['Production Cost ($/unit)'])
unitcostd70=list(double70['Production Cost ($/unit)'])
unitcostd80=list(double80['Production Cost ($/unit)'])
forecast70=list(single70['Sales Forecast(Units)'])
forecastd70=list(double70['Sales Forecast(Units)'])
forecastd800=list(double80['Sales Forecast(Units)'])

param={}
ays70={}
ayd70={}
ayd80={}

for i in range(len(months)):
    ays70[months[i]]={"Forecast" :forecast70[i] , "UnitCost" : unitcosts70[i]}
    ayd70[months[i]]={"Forecast" :forecastd70[i] , "UnitCost" : unitcostd70[i]}
    ayd80[months[i]]={"Forecast" :forecastd800[i] , "UnitCost" : unitcostd80[i]}


param["Single70"]=ays70
param["Double70"]=ayd70
param["Double80"]=ayd80
    


In [9]:
#Decision Variables

prod_rate = LpVariable.dicts("prod_rate",((month,product) for month in months for product in products),lowBound=0,cat='Integer')

regular_work = LpVariable.dicts("regular_work",((month,product) for month in months for product in products),lowBound=0,cat='Continuous')

overtime_work = LpVariable.dicts("overtime_work",((month,product) for month in months for product in products),lowBound = 0,cat='Continuous')

end_inventory = LpVariable.dicts("end_inventory",((month,product) for month in months for product in products),lowBound=0,cat='Integer')

In [10]:
#Initialize Problem

prob = LpProblem("Supply_Chain_Management",LpMinimize)

# Objective Function


In [11]:
prob += lpSum([param[prod][month]["UnitCost"]*prod_rate[(month,prod)] + param[prod][month]["UnitCost"]*month_param[month]["InventoryCarryRate"]*end_inventory[(month,prod)] + regular_work[(month,prod)]*month_param[month]["CostRegularMan"] + month_param[month]["CostRegularMan"]*overtime_work[(month,prod)]*overtime_penalty for month in months for prod in products])

In [12]:
for prod in products:
    for i in range(1,len(months)): 
        prob += end_inventory[(months[i-1],prod)] + prod_rate[(months[i],prod)] - end_inventory[(months[i],prod)] >= param[prod][months[i]]["Forecast"]

In [13]:
for prod in products:
    prob += prod_param[prod]["InitialInventory"] + prod_rate[(months[0],prod)] - end_inventory[(months[0],prod)] >= param[prod][months[0]]["Forecast"]

In [14]:
for month in months:
        prob += lpSum([regular_work[(month,prod)] for prod in products]) <= month_param[month]["AvailableManHours"]

In [15]:
for month in months:
        prob += lpSum([overtime_work[(month,prod)] for prod in products]) <= lpSum([regular_work[(month,prod)] for prod in products])*0.25

In [16]:
for month in months:
    for prod in products:
        prob += prod_rate[(month,prod)]*prod_param[prod]["StandartTime"] == regular_work[(month,prod)] + overtime_work[(month,prod)]

In [17]:
for month in months:
        prob += lpSum([end_inventory[(month,prod)] for prod in products]) <= 25000 

In [18]:
prob.solve()

1

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [20]:
value(prob.objective)

396131200.20000005

In [ ]:
prob

In [23]:
shadow_price = [{'name':name,'shadow price':c.pi, 'slack':c.slack, } for name, c in prob.constraints.items()]

In [ ]:
shadow_price

In [ ]:
prob.constraints.items()